<img src='http://hilpisch.com/tpq_logo.png' width="300px" align="right">

# FPQ Bootcamp &mdash; Day 4

**Algo Trading with Oanda**

## Imports

In [ ]:
import numpy as np
import pandas as pd
from pylab import plt
plt.style.use('seaborn')
%matplotlib inline

## configparser

In [ ]:
import configparser

In [ ]:
c = configparser.ConfigParser()

In [ ]:
c.read('fpq.cfg')

In [ ]:
c['oanda_v20']['account_id']

## The tpqoa Wrapper

In [ ]:
import tpqoa

In [ ]:
oanda = tpqoa.tpqoa('fpq.cfg')

In [ ]:
oanda = tpqoa.tpqoa('../pyalgo.cfg')

In [ ]:
# oanda.get_instruments?

In [ ]:
oanda.get_instruments()[:7]

In [ ]:
raw = oanda.get_history('DE30_EUR', '2017-1-1', '2017-3-31', 'D', 'A')
raw.head()

In [ ]:
raw = oanda.get_history('DE30_EUR', '2017-11-21', '2017-11-22', 'M1', 'A')
raw.head()

In [ ]:
raw.tail()

In [ ]:
raw['c'].plot(figsize=(10, 6));

In [ ]:
# oanda.stream_data('DE30_EUR')

In [ ]:
class OandaCollector(tpqoa.tpqoa):
    def __init__(self, *args):
        super(OandaCollector, self).__init__(*args)
        self.raw_data = pd.DataFrame()
        self.position = 0
    def on_success(self, time, bid, ask):
        print(self.ticks, end=',')
        # print('received:', time, bid, ask)
        time = pd.Timestamp(time)
        self.raw_data = self.raw_data.append(
                    pd.DataFrame({'bid': bid, 'ask': ask}, index=[time]))
        self.resam_data = self.raw_data.resample('5s', label='right').last()
        self.resam_data['mid'] = self.resam_data.mean(axis=1)
        self.resam_data['SMA1'] = self.resam_data['mid'].rolling(3).mean()
        self.resam_data['SMA2'] = self.resam_data['mid'].rolling(6).mean()
        if len(self.resam_data) > 6:
            if self.resam_data['SMA1'].iloc[-2] > self.resam_data['SMA2'].iloc[-2]:
                if self.position in [0, -1]:
                    print('going long the market')
                    self.position = 1
            if self.resam_data['SMA1'].iloc[-2] < self.resam_data['SMA2'].iloc[-2]:
                if self.position in [0, 1]:
                    print('going short the market')
                    self.position = -1

In [ ]:
oc = OandaCollector('../pyalgo.cfg')

In [ ]:
oc.stream_data('DE30_EUR', stop=75)

In [ ]:
oc.raw_data.head()

In [ ]:
oc.resam_data

<img src='http://hilpisch.com/tpq_logo.png' width="300px" align="right">